In [1]:
import socket
import numpy as np
import tensorflow as tf
import pickle
from PIL import Image
#img = Image.open('C:\\Users\\giorg\\Desktop\\workflow\\pruning\\n01582220_4014.jpeg')
# Converti l'immagine in un array NumPy
#image_array = np.array(img)

# Ridimensiona l'array alla forma desiderata (224, 224, 3)
#input_data = np.resize(image_array, (1,224, 224, 3))
#input_data = np.expand_dims(input_data)

IMAGE_SHAPE = (224, 224)
img = 'C:\\Users\\giorg\\Desktop\\workflow\\pruning\\n01582220_4014.jpeg'
input_data = Image.open(img).resize(IMAGE_SHAPE)

input_data = np.array(input_data)/255.0
input_data.shape

input_data = input_data[np.newaxis, ...]
input_data.shape


# Funzione per la serializzazione dei dati da inviare via socket
def serialize_data(data):
    serialized = pickle.dumps(data)
    return serialized

# Funzione per la deserializzazione dei dati ricevuti via socket
def deserialize_data(serialized_data):
    data = pickle.loads(serialized_data)
    return data

# Funzione per dividere la rete neurale
def split_neural_network(network, split_layer_index):
    model_client = tf.keras.models.Sequential()
    for layer in network.layers[:split_layer_index]:
        model_client.add(layer)

    model_server = tf.keras.models.Sequential()
    for layer in network.layers[split_layer_index:]:
        model_server.add(layer)

    return model_client, model_server

# Creazione della rete neurale MobileNet
mobileNet = tf.keras.applications.MobileNet()

# Definizione dell'indice di divisione della rete neurale
split_layer_index = 5

# Divisione della rete neurale
model_client, model_server = split_neural_network(mobileNet, split_layer_index)

# Connessione al server tramite socket
server_address = ("localhost", 8078)
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(server_address)

# Generazione dei dati di input per il client
#input_shape = (224, 224, 3)
#input_data = np.random.randn(*input_shape)
#input_data = np.expand_dims(input_data, 0)

# Esecuzione dell'inferenza sul modello del client
#client_output = model_client.predict(input_data)


client_output = model_client.predict(input_data)

# Serializzazione e invio del risultato al server
serialized_output = serialize_data(client_output)
data_size = len(serialized_output)
client_socket.sendall(data_size.to_bytes(8, byteorder='big'))
client_socket.sendall(serialized_output)

# Ricezione del risultato completato dal server
data_size = int.from_bytes(client_socket.recv(8), byteorder='big')
serialized_result = b''
while len(serialized_result) < data_size:
    serialized_result += client_socket.recv(4096)
result = deserialize_data(serialized_result)
print("Result:", result)

# Chiusura della connessione
client_socket.close()


Result: jay
